## Can we get data near the [Coastal Endurance](http://oceanobservatories.org/array/coastal-endurance/) array?

This notebook uses a searches the csw NGDC catalog filtering the results with:
- (a) bounding box for the Coastal Endurance array;
- (b) time span of "today &plusmn; 5 days";
- (c) variable: temperature

This approach is very robust and was used before to analyze [coastal inundation](http://nbviewer.ipython.org/github/ocefpaf/inundation_notebook/blob/master/inundation_notebook.ipynb),
check the water temperature for the [Boston Light Swim](http://nbviewer.ipython.org/github/ocefpaf/boston_light_swim/blob/master/02-create_map.ipynb) event, and even evaluate numerical ocean models performance or [skill score](http://secoora.github.io/secoora/comparisons/timeSeries/).

![](http://oceanobservatories.org/wp-content/uploads/2011/04/Endurance-Array-Map_2013_04-17_ver_0-02.jpg)

In [1]:
import os
import sys
import time
import warnings

ioos_tools = os.path.join(os.path.pardir, os.path.pardir)
sys.path.append(ioos_tools)

# Suppresing warnings for a "pretty output."
# Remove this line to debug any possible issues.
warnings.simplefilter("ignore")

start_time = time.time()

### First step: create the filter

(a) Bounding box: everything inside the dashed lines from the map above.

In [2]:
bbox = [-127, 43, -123.75, 48]

(b) Time span: today &plusmn;5 days.

In [3]:
from datetime import datetime, timedelta

dt = 5

date = datetime.utcnow()  # Uncomment to use now.
start = date - timedelta(days=dt)
stop = date + timedelta(days=dt)

In [4]:
from ioos_tools.ioos import fes_date_filter

begin, end = fes_date_filter(start, stop)

(c) Variable: `sea water temperature`.

(All names in `name_list` are [CF standard names](http://cfconventions.org/standard-names.html), but we can uses non-CF like 'Water Temperature' and that will return more datasets.)

In [5]:
sos_name = 'sea_water_temperature'

name_list = ['sea_water_temperature',
             'sea_surface_temperature',
             'sea_water_potential_temperature',
             'equivalent_potential_temperature',
             'sea_water_conservative_temperature',
             'pseudo_equivalent_potential_temperature']

Assemble the `fes` filter with `a`+`b`+`c`.

In [6]:
from owslib import fes

kw = dict(wildCard='*',
          escapeChar='\\',
          singleChar='?',
          propertyname='apiso:AnyText')

or_filt = fes.Or([fes.PropertyIsLike(literal=('*%s*' % val), **kw)
                  for val in name_list])

crs = 'urn:ogc:def:crs:OGC:1.3:CRS84'
    
filter_list = [fes.And([begin, end, fes.BBox(bbox, crs), or_filt])]

### Instantiate `csw` object using the catalogs.

Now that we have the filter we can search the catalog.
(See [OWSLib](https://geopython.github.io/OWSLib) for more information on filtering (`fes`) and the [`CatalogueServiceWeb`](https://geopython.github.io/OWSLib/#csw).)

In [7]:
from owslib.csw import CatalogueServiceWeb

catalog = 'https://dev-catalog.ioos.us/csw'
        
csw = CatalogueServiceWeb(catalog, timeout=60)
csw.getrecords2(constraints=filter_list, maxrecords=1000, esn='full')

fmt = '{:*^64}'.format
print(fmt(' Catalog information '))
print("CSW version: {}".format(csw.version))
print("Number of datasets available: {}".format(len(csw.records.keys())))

********************* Catalog information **********************
CSW version: 2.0.2
Number of datasets available: 5


### What are the services available?

In [8]:
for label, record in csw.records.items():
    services = [scheme['scheme'] for scheme in record.references]
    print('[{}]\n{}\n'.format(label, '\n'.join(set(services))))

[ncep_global]
OGC:WCS
OPeNDAP:OPeNDAP
UNIDATA:NCSS
ERDDAP:griddap
http
OGC:WMS
NOAA:LAS

[G1_SST_GLOBAL]
OPeNDAP:OPeNDAP
OGC:WCS
UNIDATA:NCSS
OGC:WMS
WWW:LINK

[CA_DAS]
OPeNDAP:OPeNDAP
OGC:WCS
UNIDATA:NCSS
OGC:WMS
WWW:LINK

[COAMPS_4KM_SOL_RAD]
OPeNDAP:OPeNDAP
OGC:WCS
UNIDATA:NCSS
OGC:WMS
WWW:LINK

[hycom_global]
OGC:WCS
OPeNDAP:OPeNDAP
UNIDATA:NCSS
ERDDAP:griddap
http
OGC:WMS
NOAA:LAS



### What do we have as `SOS` and `OPeNDAP`?

Why `SOS` and `OPeNDAP`?  Both services allow us to download and explore both the data and metadata using standard tools.
For SOS we have [pyoos](https://github.com/ioos/pyoos) (a Python library for collecting Met/Ocean observations), while for OPeNDAP we have many options:
[netCDF4-python](http://github.com/Unidata/netcdf4-python),
[iris](http://www.scitools.org.uk/iris),
[xray](https://github.com/xray/xray),
[siphon](https://github.com/Unidata/siphon)...

In [9]:
from ioos_tools.ioos import service_urls

opendap_strings = ['OPeNDAP:OPeNDAP',
                   'urn:x-esri:specification:ServiceType:odp:url']
sos_strings = ['urn:x-esri:specification:ServiceType:sos:url']

dap_urls = service_urls(csw.records, services=opendap_strings)
sos_urls = service_urls(csw.records, services=sos_strings)

if sos_urls:
    print(fmt(' SOS '))
    for url in dap_urls:
        print('{}'.format(url))
if dap_urls:
    print(fmt(' DAP '))
    for url in dap_urls:
        print('{}.html'.format(url))

***************************** DAP ******************************
http://oos.soest.hawaii.edu/thredds/dodsC/hioos/model/atm/ncep_global/NCEP_Global_Atmospheric_Model_best.ncd.html
http://oos.soest.hawaii.edu/thredds/dodsC/pacioos/hycom/global.html
http://thredds.cencoos.org/thredds/dodsC/CA_DAS.nc.html
http://thredds.cencoos.org/thredds/dodsC/COAMPS_4KM_SOL_RAD.nc.html
http://thredds.cencoos.org/thredds/dodsC/G1_SST_GLOBAL.nc.html


### Get the SOS data with pyoos `collectors`.

In [10]:
from pyoos.collectors.ndbc.ndbc_sos import NdbcSos
from pyoos.collectors.ioos.swe_sos import IoosSweSos
from pyoos.collectors.coops.coops_sos import CoopsSos

from ioos_tools.ioos import collector2table

observations = []
for url in sos_urls:
    if 'co-ops' in url.lower():
        collector = CoopsSos()
    elif 'ndbc' in url.lower():
        collector = NdbcSos()
    else:
        collector = IoosSweSos(url)
    
    collector.set_bbox(bbox)
    collector.end_time = stop
    collector.start_time = start
    collector.variables = [sos_name]
    
    ofrs = collector.server.offerings
    title = collector.server.identification.title
    try:
        observations.extend(collector2table(collector))
        print('{}: {} offerings:'.format(title, len(ofrs)))
        print(fmt(' {} '.format(url)))
    except Exception as e:
        print('\nCannot collect:\n{}. {}'.format(url, e))

In [11]:
observations

[]